In [ ]:
import pose_sequence_loader
import importlib
importlib.reload(pose_sequence_loader)

from pose_sequence_loader import getRecordingFolderMapping

getRecordingFolderMapping(recordings, "/dhc/groups/bp2021ba1/alex/UnicornML/src")


In [ ]:
def findPoseFile(recording_activities, pose_file_metadata_tuples):
    for pose_file, pose_activites, startTimestamp in pose_file_metadata_tuples:
        if len(recording_activities) != len(pose_activites):
            continue
        for i, recording_activity in enumerate(recording_activities):
            if recording_activity != pose_activites[i] and pose_activites[i] != -1:
                continue
            if i == len(recording_activities) - 1: 
                return pose_file, startTimestamp

    return None, None

In [ ]:
def deleteConsecutives(series):
    filtered_items = []
    last_item = None
    for i, item in series.iteritems():
        if item != last_item:
            filtered_items.append(item)
        last_item = item
    return filtered_items

def getPoseFilesWithMetadata(dataset_path: str, activity_mapping = None):
    from utils.file_functions import get_subfolder_names
    import json
    import os
    import pandas as pd

    recording_folder_names = get_subfolder_names(dataset_path)
    recording_folder_names = [
        os.path.join(dataset_path, recording_folder_name)
        for recording_folder_name in recording_folder_names
    ]
    pose_file_metadata_tuples = []
    for recording_folder_path in recording_folder_names:
        for file_name in os.listdir(recording_folder_path):
            is_file = os.path.isfile(os.path.join(recording_folder_path, file_name))
            if is_file and file_name.endswith("poseSequence.csv"):
                pose_file_name = os.path.join(recording_folder_path, file_name)
            if is_file and file_name.endswith("metadata.json"):
                metadata_file_name = os.path.join(recording_folder_path, file_name)
            if is_file and file_name.endswith(".csv") and not file_name.endswith("poseSequence.csv"):
                sensor_file_name = os.path.join(recording_folder_path, file_name)

        try:
            sensor_frame = pd.read_csv(
                sensor_file_name, skiprows=8
            )

            with open(metadata_file_name) as metadata_file:
                metadata = json.load(metadata_file)
            
            activities = list(map(
                lambda activityDict: activityDict["label"],
                metadata["activities"]
            ))

            if activity_mapping != None: 
                activities = list(map(
                    lambda activity: -1 if activity not in activity_mapping.keys() else activity_mapping[activity],
                    activities
                ))
            metadata = {
                "activities": activities,
                "startTimestamp": metadata["startTimestamp"],
                "subject": metadata["person"],
                "numLines": sensor_frame.shape[0],

            }
            pose_file_metadata_tuples.append((pose_file_name, metadata))
        except:
            raise Exception(f"Recording with missing metadata or poseSequence found: {recording_folder_path}")
    return pose_file_metadata_tuples


def findPoseFile(reocording: Recording, pose_file_metadata_tuple):
    recording_activities = deleteConsecutives(recording.activities)
    metrics = []
    for pose_file, metadata in pose_file_metadata_tuples:
        metricsDict = {}
        metricsDict["pose_file"] = pose_file
        metricsDict["metadata"] = metadata

        metricsDict["activityLengthDiff"] = abs(len(metadata["activities"]) - len(recording_activities))
        metricsDict["activityDiff"] = len([act for act in set(recording_activities) if act not in metadata["activities"]])
        metricsDict["numLineDiff"] = abs(recording.time_frame.shape[0] - metadata["numLines"])
        metricsDict["sameSubject"] = recording.subject[:-4] == metadata["subject"]

        metrics.append(metricsDict)

    subject_metrics = list(filter(lambda metric: metric["sameSubject"], metrics))

    minALD = min([metric["activityLengthDiff"] for metric in subject_metrics])
    minAD = min([metric["activityDiff"] for metric in subject_metrics])
    minNLD = min([metric["numLineDiff"] for metric in subject_metrics])

    for metric in metrics:
        metric["score"] = 3 if metric["sameSubject"] else 0
        metric["score"] += (1 if metric["activityLengthDiff"] == minALD else 0)
        metric["score"] += (1 if metric["activityDiff"] == minAD else 0)
        metric["score"] += (1 if metric["numLineDiff"] == minNLD else 0)

    sortedMetrics = sorted(metrics, key=lambda metric: metric["score"], reverse=True)

    return sortedMetrics[0]

In [ ]:
def getRecordingFolderMapping(recordings: 'list[Recording]', path: str) -> None:
    import os 
    import json

    if not os.path.exists(path):
        os.makedirs(path)

    mapping = {}
    for (index, recording) in enumerate(recordings):

        recording.activities.index = recording.sensor_frame.index
        filename = str(index) + '_' + recording.subject + '.csv'

        mapping[filename] = recording.recording_folder
    
    with open(f"{path}/recording_folder_map.json", "w+") as file:
        json.dump(mapping, file, indent=4)
    
    print(f'Saved recording_folder_map.json')

In [ ]:
def indexToLabel(index):
    category_labels = {'rollstuhl transfer': 0, 'essen reichen': 1, 'umkleiden': 2, 'bad vorbereiten': 3, 'bett machen': 4, 'gesamtwaschen im bett': 5, 'aufräumen': 6, 'geschirr einsammeln': 7, 'essen austragen': 8, 'getränke ausschenken': 9, 'küchenvorbereitung': 10, 'waschen am waschbecken': 11, 'rollstuhl schieben': 12, 'mundpflege': 13, 'haare kämmen': 14, 'essen auf teller geben': 15, 'dokumentation': 16, 'aufwischen (staub)': 17, 'haare waschen': 18, 'medikamente stellen': 19, 'accessoires anlegen': 20, 'föhnen': 21, 'null - activity': 22, 'ikp versorgung': 23}
        
    for (label, i) in category_labels.items():
        if i == index:
            return label
    return "-1"
    

print(f"Frame: {frame_diff}       {'>' if frame_diff > pose_diff else '<'}            Pose: {pose_diff}")
frame_activities = deleteConsecutives(activities)
try:
    print(f"{indexToLabel(frame_activities[0])}, {indexToLabel(frame_activities[1])}         {pose_activities[0]['label']}, {pose_activities[1]['label']}")
except:
    print("//////////////////////////")